# pytwoway example

In [1]:
# Add pytwoway to system path, do not run this
# import sys
# sys.path.append('../../..')

In [2]:
# Import the pytwoway package 
# (Make sure you have installed it using pip install pytwoway)
import pytwoway as tw

## Simulate some data

The package contains functions to simulate data. We use this here to keep things simple. If you have your own data, you can import it. Load it as a pandas dataframe and use it as an input. 

As you can see, we will need the following required columns in our data: 

 - `wid`: the worker identifier
 - `fid`: the firm identifier
 - `year`: the time
 - `comp`: the outcome variable, in our case compensation

In [3]:
# For the example, we simulate data
sim_data = tw.SimTwoWay().sim_network()
display(sim_data)

,wid,year,k,alpha,psi,spell,freq,fid,move,comp
0,1,1,4,0.430727,-0.114185,1,1,62,False,-0.002648
1,1,2,2,0.430727,-0.604585,2,1,41,True,0.081698
2,1,3,2,0.430727,-0.604585,3,1,35,True,-0.671860
3,1,4,6,0.430727,0.348756,4,1,107,True,-1.581872
4,1,5,4,0.430727,-0.114185,5,1,78,True,-0.340476
...,...,...,...,...,...,...,...,...,...,...
49995,10000,1,4,0.430727,-0.114185,1,2,77,False,0.608855
49996,10000,2,4,0.430727,-0.114185,1,2,77,False,1.941444
49997,10000,3,6,0.430727,0.348756,2,3,120,True,-0.267260
49998,10000,4,6,0.430727,0.348756,2,3,120,False,0.504194


## Create a TwoWay object using your data

In [4]:
# We need to specify a column dictionary to make sure columns are named correctly
# You can also manually update column names yourself
col_name_dict = {
    'fid': 'fid',    # Specify the column name for the firm identifier 
    'wid': 'wid',    # Specify the column name for the worker identifier 
    'year': 'year',  # Specify the column name for the year
    'comp': 'comp'   # Specify the column name for the outcome variable
}

# Create the TwoWay object that will do all the heavy lifting
tw_net = tw.TwoWay(data=sim_data, formatting='long', col_dict=col_name_dict)

## Now we can run the FE estimator

In [5]:
## Optional Parameters ##
fe_params = {
    'ncore': 1, # Number of cores to use
    'batch': 1, # Batch size to send in parallel
    'ndraw_pii': 50, # Number of draw to use in approximation for leverages
    'ndraw_tr': 5, # Number of draws to use in approximation for traces
    'check': False, # Whether to compute the non-approximated estimates as well
    'hetero': False, # Whether to compute the heteroskedastic estimates
    'out': 'res_fe.json', # Filepath for fe results
    'con': False, # Computes the smallest eigen values, this is the filepath where these results are saved
    'logfile': '', # Log output to a logfile
    'levfile': '', # File to load precomputed leverages
    'statsonly': False # Save data statistics only
}

# Estimate the fixed effect decomposition
tw_net.fit_fe(user_fe=fe_params)

100%|██████████| 5/5 [00:00<00:00, 217.92it/s]


## We can also run the CRE estimator

In [6]:
## Optional Parameters ##
cre_params = {
    'ncore': 1, # Number of cores to use
    'ndraw_tr': 5, # Number of draws to use in approximation for traces
    'ndp': 50, # Number of draw to use in approximation for leverages
    'out': 'res_cre.json', # Filepath for cre results
    'posterior': False, # Whether to compute the posterior variance
    'wobtw': False # Sets between variation to 0, pure RE when computing cre
}
KMeans_params = { # These parameters are specifically for the KMeans algorithm
                  # Read more at https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
    'n_clusters': 10,
    'init': 'k-means++',
    'n_init': 500,
    'max_iter': 300,
    'tol': 0.0001,
    'precompute_distances': 'deprecated',
    'verbose': 0,
    'random_state': None,
    'copy_x': True,
    'n_jobs': 'deprecated',
    'algorithm': 'auto'
}
cluster_params = {
    'cdf_resolution': 10, # How many values to use to approximate the cdf when clustering
    'grouping': 'quantile_all',
        # How to group the cdfs when clustering
        # ('quantile_all' to get quantiles from entire set of data,
        #     then have firm-level values between 0 and 1;
        # 'quantile_firm_small' to get quantiles at the firm-level
        #     and have values be compensations if small data;
        # 'quantile_firm_large' to get quantiles at the firm-level
        #     and have values be compensations if large data,
        #     note that this is up to 50 times slower than 'quantile_firm_small'
        #     and should only be used if the dataset is too large to copy
        #     into a dictionary)
    'year': None, # If None, uses entire dataset when clustering;
                  # If int, gives year of data to consider when clustering
    'user_KMeans': KMeans_params
}

# Estimate the cre model
tw_net.fit_cre(user_cre=cre_params, user_cluster=cluster_params)

## Finally, we can investigate the results

In [7]:
display(tw_net.summary_fe())
display(tw_net.summary_cre())

{'var_y': 1.9850874905837954,
 'var_fe': 0.5877239062756846,
 'cov_fe': 0.17524843068460458,
 'var_ho': 0.5822717184260641,
 'cov_ho': 0.1771904511905588}

{'var_y': 1.9850874905837954,
 'var_bw': 0.5845515792925647,
 'cov_bw': 0.17582443706921494,
 'var_tot': 0.5847032595847196,
 'cov_tot': 0.17571711821397532}